In [5]:
from ultralytics import YOLO
import torch

In [11]:
model = YOLO("yolov8n-seg.yaml")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

results = model.train(data = "/home/nirshal/codes/Dataset-generator-using-SAM/my_custom_dataset/yolo_seg.yaml", epochs = 100, imgsz = 640)


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [12]:
model2 = YOLO("/home/nirshal/codes/Dataset-generator-using-SAM/runs/segment/train2/weights/best.pt")
results = model2(["/home/nirshal/Downloads/Water Bottles.v3-water-bottles.yolov8/valid/images/plastic_bottle-8_jpg.rf.d8d39b23fad6f3d83278a67bf6b1ec27.jpg", "/home/nirshal/Downloads/Water Bottles.v3-water-bottles.yolov8/valid/images/plastic_bottle-90_jpg.rf.e35a98ef6d43e810b9dc7f9f862e59c6.jpg"])



/home/nirshal/anaconda3/envs/work/lib/python3.10/site-packages/torch/nn/modules/conv.py:952: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv_transpose2d(
0: 640x640 3 bottles, 1: 640x640 (no detections), 77.9ms
Speed: 2.8ms preprocess, 38.9ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 640)


In [16]:
import cv2
import numpy as np

for i, result in enumerate(results):
    # Load the original image
    img = cv2.imread(result.path)
    
    # Get the original image dimensions
    height, width = img.shape[:2]
    
    # Draw the segmentation masks
    if result.masks is not None:
        for mask in result.masks.data.cpu().numpy():
            # Resize the mask to match the image dimensions
            mask = cv2.resize(mask, (width, height))
            mask = (mask * 255).astype(np.uint8)
            colored_mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
            img = cv2.addWeighted(img, 0.7, colored_mask, 0.3, 0)
    
    # Draw the bounding boxes
    if result.boxes is not None:
        for box in result.boxes.data.cpu().numpy():
            x1, y1, x2, y2 = map(int, box[:4])
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Save the result
    cv2.imwrite(f"detailed_result_{i}.jpg", img)